<a href="https://colab.research.google.com/github/sanikac10/pandas/blob/main/Assignment_3_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 3: Decoder-only Transformer

Hello! Welcome to your last assignment, where you will be asked to implement a decoder-only transformer model, and train it on next word prediction objectives.

Because this is a GPU-based assignment, you are encouraged to use Google Colab's T4 runtime. It contains a 15G GPU, which we have tested to be enough for this assignment. However, there is a limited amount of free T4 usage per day. We encourage you to only connect to a runtime when you want to run the code.

You are free to make minor changes to the provided code if that will make things eaiser. However, you will be asked to summarize what you changed in the final section.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

First, we will define some hyperparameters. You do not need to change these hyperparameters.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datasets import load_dataset
import re
from tqdm import tqdm
from collections import Counter
from torch.optim.lr_scheduler import LambdaLR

# Hyperparameters
vocab_size = 32100
embedding_dim = 256
num_heads = 8
num_layers = 8
ffn_dim = 256
max_seq_len = 256
batch_size = 64
num_epochs = 1
learning_rate = 0.001
unk_token = "<UNK>"
pad_token = "<PAD>"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
dataset = load_dataset("alpindale/light-novels", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

LightNovels.txt:   0%|          | 0.00/907M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9240994 [00:00<?, ? examples/s]

In [ ]:
dataset["text"]

['',
 ' ',
 '“RISHE IRMGARD WEITZNER! You are a vile woman! A truly malicious creature unworthy of the crown prince! As of this instant, our engagement is off!” ',
 '“As you wish, Your Highness.”',
 '“…Huh?”',
 'The prince’s declaration echoed throughout the ballroom. Rishe, the duke’s daughter, bowed in response, her coral-toned hair swaying gracefully. The assembled guests found themselves captivated by her dignified beauty. Despite having just witnessed a broken engagement, they couldn’t bring themselves to pity her.',
 'Slowly, Rishe lifted her head and fixed the prince with a pale-emerald gaze. It caught him off guard, but he quickly recovered.',
 '“Didn’t you hear what I just said?! I’m calling off our engagement! Doesn’t that trouble you?”',
 '“Not really.”',
 'Rishe already knew what happened next: false accusations, exile, and her family severing all ties and leaving her to survive on her own. She turned from the prince. There was no point wasting any time.',
 'This is my seve

## Section 1: Implement the Decoder-only Transformer.

In [ ]:
from transformers import AutoTokenizer

# Tokenizer
# We will use a pre-trained tokenizer from T5-base, which is why vocab_size is 32100 above.
# You will need to research on how huggingface tokenizers work
# https://huggingface.co/docs/transformers/en/main_classes/tokenizer
class BPETokenizer:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base")

    def encode(self, text):
        # TODO
        # Implement this encode function that return a list of INTs (word IDs) based on the input text (str)

    def decode(self, token_ids):
        # TODO
        # Implement this decode function that returns a string based on a list of INTs (word IDs)

# Positional Encoding
# TODO: You need to implement a sinusoidal positional embedding class.
# There are many great resources, such as https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/
# Input: current word embedding
# Output: new word embedding after adding the positional embeddings.
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, max_len):
        super().__init__()
        # TODO

    def forward(self, x):
        # TODO

# Self-Attention Layer
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super().__init__()
        assert embedding_dim % num_heads == 0
        # TODO
        # Hint: you need the QKV weights, also some bookkeeping values.
        # Don't forget about a fully-connected output layer

    # Attention masks are values of 1's and 0's.
    # When the attention mask for a position is 1, it means that the attention can be computed on that position.
    # If a mask value for a position is 0, it means that the attention should 'skip' or 'mask-out' that position.
    def forward(self, x, mask=None):
        # TODO
        # Implement the computations of attention scores, with respect to the attentin mask 'mask'
        # Hint: consider something like attention_scores.masked_fill(?, ?) to compute the masked scores.

# Transformer Decoder Layer
class TransformerDecoderLayer(nn.Module):
    def __init__(self, embedding_dim, num_heads, ffn_dim):
        super().__init__()
        self.self_attn = MultiHeadSelfAttention(embedding_dim, num_heads)
        self.norm1 = nn.LayerNorm(embedding_dim)
        # TODO
        # Implement a MLP layer here.
        # Research about what nn.LayerNorm is doing here.
        # Add a second layernorm after the MLP.

    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, mask)
        # TODO: research about why it is x + attn_output, instead of only attn_output?
        x = self.norm1(x + attn_output)
        # TODO: finish the rest.

# Decoder-only Transformer Model
class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, num_layers, ffn_dim, max_seq_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_encoding = PositionalEncoding(embedding_dim, max_seq_len)
        # TODO
        # Implement the layers

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        # TODO
        # Create an attention mask, and do the rest of the computation.

## Section 2: Implement the LLM training.

In [ ]:
def load_training_corpus():
    dataset = load_dataset("alpindale/light-novels", split="train")
    texts = dataset["text"][:3000000]
    tokenizer = BPETokenizer()
    tokenized = []
    # TODO
    # In pre-training, we will make every training sequence with exact same lengths (max_seq_len)
    # This means that you will to concatenate shorter text in 'texts' to make every list in 'tokenized' exactly max_seq_len
    # In other words, for instance in tokenized: assert len(instance) == max_seq_len.
    # As you can see later, each element in tokenized is a number (word IDs).
    # For the final instance where you don't have enough tokens, you can pad the sequence with word id 1.

    return torch.tensor(tokenized, dtype=torch.long), tokenizer

data, tokenizer = load_book_corpus()
train_data = data

def get_lr_schedule(optimizer, warmup_steps, total_steps):
    """
    Creates a learning rate schedule with linear warmup and cosine decay.
    """
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            return current_step / warmup_steps  # Linear warmup
        progress = (current_step - warmup_steps) / (total_steps - warmup_steps)
        return 0.5 * (1 + torch.cos(torch.tensor(progress * 3.1415926535)))  # Cosine decay

    return LambdaLR(optimizer, lr_lambda)

def train():
    model = DecoderOnlyTransformer(vocab_size, embedding_dim, num_heads, num_layers, ffn_dim, max_seq_len).to(device)
    # TODO: compute the model size. You can use a programmatic approach.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    warmup_steps = int(0.1 * (num_epochs * len(train_data) // batch_size))  # 10% of total steps
    total_steps = num_epochs * (len(train_data) // batch_size)
    scheduler = get_lr_schedule(optimizer, warmup_steps, total_steps)

    for epoch in range(num_epochs):
        total_loss = 0
        progress_bar = tqdm(range(0, len(train_data), batch_size), desc=f"Epoch {epoch+1}")
        for i in progress_bar:
            batch = train_data[i:i+batch_size].to(device)
            # TODO Implement the training process.
            inputs = # TODO
            targets = # TODO. What is supposed to be the supervision targets in autogressive LM?

            # Your code should finish before scheduler.step() here.
            scheduler.step()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())
        print(f"Epoch {epoch+1} completed with Loss: {total_loss / len(train_data)}")
    return model


Now, let's train the model! We suggest you save the trained model for later testing.

In [ ]:
model = train()
torch.save(model.state_dict(), "model.ckpt")

Then, let's evaluate your model to see if it's working!

In [ ]:
# We have provided a sample greedy decoding generation function for you.
# You can use it to test if your trained model is working.
# Although we are not providing any actual test cases, it will be pretty easy
# to tell if your language model is generating coherent natural languages.
# Try a few examples! If it works, it works; we will not do 'exact-match' grading.
model.eval()
def generate_text_greedy(model, input_text, max_length=20):
    input_tokens = tokenizer.encode(input_text)
    generated = input_tokens[:]
    for _ in range(max_length - len(input_tokens)):
        input_tensor = torch.tensor([generated], device=device)
        with torch.no_grad():
            output = model(input_tensor)
        next_token = torch.argmax(output[:, -1, :], dim=-1).item()
        generated.append(next_token)
        if next_token == 1:
            break
    return tokenizer.decode(generated)

print("Generated Sequence:", generate_text_greedy(model, "what is your"))

def generate_text_topk(model, input_text, max_length=20, k=50, temperature=1.0):
    # TODO
    # Implement a top-k sampling generation function.
    # Steps
    # 1. adjust the output distribution/logits based on the temperature
    # 2. find the top-k next tokens (hint: torch.topk)
    # 3. readjust the probability for the top-k candidates for them to sum to 1 (hint: softmax)
    # 4. sample a word from the new k tokens (hint: torch.multinomial)

print("Generated Sequence:", generate_text_topk(model, "what is your"))

## Section 3: Written questions.
Answer all questions with ~100 words (use the fewest words that are enough to express your meanings)

### 1. What does nn.LayerNorm do?

### 2. Why the provided code use self.norm1(x + attn_output) instead of self.norm1(attn_output)?

### 3. What is the model size using the provided hyperparameters?

### 4. How does temperature play a part in top-k sampling? Share some observations.

### 5. (Optional, not graded) Please summarize your change to the provided code, if any.
